### Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import constants, utils

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

os.environ["OPENAI_API_KEY"] = utils.get_openai_api_key()

### Using Language Models

Language Models do not have inherent memory

In [3]:
from langchain_core.messages import HumanMessage
model = ChatOpenAI(model="gpt-3.5-turbo")
model.invoke([HumanMessage(content="Hi! I'm Bob")])
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not have access to that information.", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 12, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-45e67098-0b24-4808-b1f9-0ec8748fe2c2-0', usage_metadata={'input_tokens': 12, 'output_tokens': 13, 'total_tokens': 25})

Resend all messages to get around this

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3fae2ef6-21ad-4895-b0a4-5580a77646cc-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})

### Message History

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [7]:
config = {"configurable": {"session_id": "abc2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

Parent run 3972d903-f6aa-4c2d-b4b0-8c6af9f146df not found for run 5a5b99b9-83c8-48df-bf38-46e519f1aa3b. Treating as a root run.


'Hello Bob! How can I assist you today?'

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run d3e6c432-c3c9-4c52-8845-26e4975c2f0a not found for run 97927899-8d35-4499-8380-99ee035756ba. Treating as a root run.


'Your name is Bob. How can I assist you today, Bob?'

In [9]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run b711d1b5-3828-4b53-a139-c4aa66132089 not found for run dd1bfedd-cd20-44b1-b93f-1837d9518cf5. Treating as a root run.


"I'm sorry, but I don't know your name. Can you please tell me?"

In [10]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 64f02b90-60e4-4a69-a3fd-1042feb1359d not found for run 244b613c-089c-4bd3-bf63-0305c37def2d. Treating as a root run.


'Your name is Bob. How can I assist you today, Bob?'

### Prompt Templates

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [12]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello Bob! How can I assist you today?'

In [13]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc5"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

Parent run 84c78230-c425-4310-a777-e47e66808a5f not found for run b783a774-e8a2-4f94-9bfa-fe0e1fc669d5. Treating as a root run.


'Hello, Jim! How can I assist you today?'

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 82c25c38-449d-4772-9e63-25756e36ba16 not found for run 820a15c3-c648-4b63-b24b-e4ee0300997b. Treating as a root run.


'Your name is Jim.'

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [16]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola Bob! ¿En qué puedo ayudarte hoy?'

In [17]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc11"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

Parent run 6adb7c3f-f54c-42ff-ba09-6ff516d68587 not found for run bc7cf9e9-8ae8-40e9-9d61-312f6b1e424e. Treating as a root run.


'¡Hola, Todd! ¿En qué puedo ayudarte hoy?'

In [18]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

Parent run 856701e5-2e9d-479b-bd57-af384e04a3c7 not found for run 9533515a-85cc-43f0-bf7e-08d99c77dcf2. Treating as a root run.


'Tu nombre es Todd. ¿Hay algo más en lo que pueda ayudarte?'

### Managing Conversation History

In [19]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [20]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I can't see your name. How can I assist you today?"

In [21]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

'Your favorite ice cream flavor is vanilla.'

In [22]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run fba994db-b0c6-432e-8105-4c9f6bb3b7db not found for run a4d6bdbc-491d-4d99-8762-03170ae7edfa. Treating as a root run.


"I'm sorry, I don't have access to your name. How can I assist you today?"

In [23]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 815840f3-a0d5-47ac-aaef-345550019b0f not found for run fbe8467c-6ddb-4820-9af6-f1ac25a71bd5. Treating as a root run.


"I'm sorry, I don't have that information. What else would you like to know?"